# Scrape Data

The purpose of this notebook is to extract records from the raw data into a workable unified format.

It is sorta Named Entity Resolution (NER), but I'm not sure if it academically neatly falls into this category as it is more of a rudimentary preprocessing step, more similar to document intelligence.

No deduplication / entity resolution is performed within this step. See Notebook 4 for deduplication efforts.

## Load raw data into memory

Load the files using f# as I'm not sure how to load files in javascript

NOTE: Potential improvement is to not store all the files content in memory at once. However, that's a problem to be bridged when it becomes a problem.

In [2]:
open System.IO

let directory = "../data/raw"
let searchPattern = "*.md"

let markdownFiles =
    Directory.EnumerateFiles(directory, searchPattern, SearchOption.AllDirectories)
    // Excluding licenses as they are not part of the data
    |> Seq.filter (fun filePath -> not (Path.GetFileName(filePath).ToLowerInvariant() = "license.md"))

let fileDict = new System.Collections.Generic.Dictionary<string, string>()
for filePath in markdownFiles do
    let fileContent = File.ReadAllText(filePath)
    fileDict.[Path.GetFileName(filePath)] <- fileContent

fileDict

key value awesome-fsharp.md # <img src="http://fsprojects.github.io/assets/logo.png" width="26"> Awesome F# #

[![Awesome](https://cdn.rawgit.com/sindresorhus/awesome/d7305f38d29fed78fa85652e3a63e154dd8e8829/media/badge.svg)](https://github.com/sindresorhus/awesome)
[![Join the chat at https://gitter.im/VPashkov/awesome-fsharp](https://badges.gitter.im/VPashkov/awesome-fsharp.svg)](https://gitter.im/VPashkov/awesome-fsharp?utm_source=badge&utm_medium=badge&utm_campaign=pr-badge&utm_content=badge)

A curated list of awesome F# frameworks, libraries, software and resources.

## Table of Contents
- [Awesome F#](#)
 - [F# wrappers for popular .NET libraries](#f-wrappers-for-popular-net-libraries)
 - [Actor frameworks](#actor-frameworks)
 - [Build tools](#build-tools)
 - [Cloud](#cloud)
 - [Compilers](#compilers)
 - [Concurrent, asynchronous and parallel programming](#concurrent-asynchronous-and-parallel-programming)
 - [Configuration](#configuration)
 - [Data Science](#data-science)
 - [Development Tools](#development-tools)
 - [IDE](#ide)
 - [Editor plugins](#editor-plugins)
 - [Performance analysis](#performance-analysis)
 - [Game development](#game-development)
 - [General purpose libraries](#general-purpose-libraries)
 - [GUI](#gui)
 - [HTTP Clients](#http-clients)
 - [Logging](#logging)
 - [Package Management](#package-management)
 - [Parsing](#parsing)
 - [PreCompilation](#precompilation)
 - [Search](#search)
 - [Serialization](#serialization)
 - [Simulation](#simulation)
 - [Testing](#testing)
 - [Type providers](#type-providers)
 - [Creating type providers](#creating-type-providers)
 - [Visualization](#visualization)
 - [Web frameworks](#web-frameworks)
 - [.Net Core Templates](#net-core-templates)
 - [Resources](#resources)
 - [Blogs](#blogs)
 - [Cheatsheets](#cheatsheets)
 - [Community](#community)
 - [Other Lists](#other-lists)
 - [Websites](#websites)
 - [Videos](#videos)
 - [Courses](#courses)

## F# wrappers for popular .NET libraries
Looking to have a more enjoyable experience when consuming a popular .NET library? Here is a quick table.

.NET Library | F# Wrapper
-|-
[Avalonia](https://github.com/AvaloniaUI/Avalonia) | [Avalonia.FuncUI](https://github.com/fsprojects/Avalonia.FuncUI)
[ASP.NET Core](https://github.com/dotnet/aspnetcore) | [Giraffe](https://github.com/giraffe-fsharp/Giraffe) (+ optionally [Saturn](https://github.com/SaturnFramework/Saturn))
[ASP.NET Core Blazor](https://github.com/dotnet/aspnetcore/tree/main/src/Components) | [Bolero](https://github.com/fsbolero/Bolero)
[MSTest](https://github.com/microsoft/testfx)/[NUnit](https://github.com/nunit/nunit)/[xUnit.net](https://github.com/xunit/xunit) | [FsUnit](https://github.com/fsprojects/FsUnit)
[System.Text.Json](https://github.com/dotnet/runtime/tree/main/src/libraries/System.Text.Json) | [FSharp.SystemTextJson](https://github.com/Tarmil/FSharp.SystemTextJson)
[WPF](https://github.com/dotnet/wpf) | [Elmish.WPF](https://github.com/elmish/Elmish.WPF)
[Xamarin.Forms](https://github.com/xamarin/Xamarin.Forms) | [Fabulous](https://github.com/fabulous-dev/Fabulous)

## Actor frameworks

* **[Akka.NET ★ 2239 ⧗ 0](https://github.com/akkadotnet/akka.net)** - Community-driven port of the popular Java/Scala framework Akka to .NET. [Apache 2.0]
* [Akkling ★ 45 ⧗ 1](https://github.com/Horusiath/Akkling) - F# typed API for Akka.NET. [Apache 2.0]
* [Cricket ★ 141 ⧗ 380](https://github.com/fsprojects/Cricket) - Actor framework for F#. [Unlicense]
* [Orleankka ★ 175 ⧗ 5](https://github.com/OrleansContrib/Orleankka) - Functional API for Orleans Framework. [Apache 2.0]
* **[Orleans ★ 2754 ⧗ 0](https://github.com/dotnet/orleans)** - Distributed Virtual Actor Model. [MIT]
* **[Proto.actor ★ 692 ⧗ 0](https://github.com/AsynkronIT/protoactor-dotnet)** - Cross-platform actor framework for .NET, GO, JAVA and KOTLIN. [Apache 2.0]


## Build tools

* **[FAKE ★ 733 ⧗ 0](https://github.com/fsharp/FAKE)** - "F# Make" is a cross platform build automation system. [Apache 2.0]
* **

## Extract entities

In this step, the entities are going to be extracted from the raw data. But first, we need to setup the environment

### Define some javascript resources

Imports and helper functions to make coding in javascript easier inside of this notebook.

The circular replacer makes it so that javascript objects can be serialized even when they are recursive, which helps with debugging.
Using cheerio and marked, both javascript packages, parse the content into a queryable tree.

In [3]:
// Imports
marked = await import("https://cdn.jsdelivr.net/npm/marked/lib/marked.esm.js");
cheerio = await import("https://cdn.jsdelivr.net/npm/cheerio@1.0.0-rc.12/+esm");

// Helper Functions
getCircularReplacer = function (maxDepth) {
    const ancestors = [];
    let depth = 0;
    return function (key, value) {
      if (depth > maxDepth) {
        return "[Max depth reached]";
      }
      if (typeof value !== "object" || value === null) {
        return value;
      }
      // `this` is the object that value is contained in,
      // i.e., its direct parent.
      while (ancestors.length > 0 && ancestors.at(-1) !== this) {
        ancestors.pop();
        depth--;
      }
      if (ancestors.includes(value)) {
        return "[Circular]";
      }
      ancestors.push(value);
      depth++;
      return value;
    };
  };

### Perform entity extraction

Using marked and cheerio, extracting entities from markdown files.
Following code extracts headers, name, description and link from the markdown document. Records are grouped by filename so that source attribution is preserved.

In [4]:
#!set --value @fsharp:fileDict --name fileDict

//console.log(Object.entries(fileDict).map(x => x.length));
results = {};

for (const entry of Object.entries(fileDict)) {
    const [filename, content] = entry;
    
    console.log(filename);
    const $ = cheerio.load(marked.parse(content));

    let unorderedLists = {};
    // Find all unordered lists that are not nested inside of another unordered list (Top Level)
    $('ul:not(ul ul)').each(function(i, elem) {
        const unorderedListNode = $(this);
        unorderedLists[$(this)] = [];
    
        // Find all list items inside of the unordered list
        $(this).find('li').each(function(i, elem) {
            unorderedLists[unorderedListNode].push($(this));
        });
    });

    const result = Object.keys(unorderedLists).map(key => unorderedLists[key].map(li => {
        const link = $(li).find('a').attr('href');
        const description = $(li).text();
        const name = $(li).find('a').text();

        // Find all headers that are above the list item
        let headers = [];
        for (let i = 6; i >= 1; i--) {
            const header = $(li).parents().prevAll(`h${i}`).first();
            if (header.length) {
                headers.push(header.text());
            }
        }
        return { link, description, name, headers };
    })).flat();

    results[filename] = result;
}

console.log(JSON.stringify(results, null, 2));

results = Object.entries(results).map(x => {
    return {
        filename: x[0],
        data: x[1]
    };
});


awesome-fsharp.md

awesome-knowledge-graph.md

{
  "awesome-fsharp.md": [
    {
      "link": "#",
      "description": "Awesome F#\nF# wrappers for popular .NET libraries\nActor frameworks\nBuild tools\nCloud\nCompilers\nConcurrent, asynchronous and parallel programming\nConfiguration\nData Science\nDevelopment Tools\nIDE\nEditor plugins\nPerformance analysis\n\n\nGame development\nGeneral purpose libraries\nGUI\nHTTP Clients\nLogging\nPackage Management\nParsing\nPreCompilation\nSearch\nSerialization\nSimulation\nTesting\nType providers\nCreating type providers\n\n\nVisualization\nWeb frameworks\n.Net Core Templates\nResources\nBlogs\nCheatsheets\nCommunity\nOther Lists\nWebsites\nVideos\nCourses\n\n\n\n",
      "name": "Awesome F#F# wrappers for popular .NET librariesActor frameworksBuild toolsCloudCompilersConcurrent, asynchronous and parallel programmingConfigurationData ScienceDevelopment ToolsIDEEditor pluginsPerformance analysisGame developmentGeneral purpose librariesGUIHTTP ClientsLoggingPackage ManagementParsingPreCompil

## Save output to file

Now that the entities have been extracted from the markdown files into a more useful unified dataset of entities found within each file, save the output of the scraping step to a single json file.

I don't know how to access io in javascript, so going back to f#.

In [5]:
#!set --value @javascript:results --name results
open System.IO
open System.Text.Json

let filePath = "../data/scrapped/scrapped-dataset.json"
let json = JsonSerializer.Serialize(results, JsonSerializerOptions(WriteIndented = true))

let directoryPath = Path.GetDirectoryName(filePath)
if not (Directory.Exists(directoryPath)) then // Equiv to not <| Directory.Exists(directoryPath), looks like a technique to remove a layer of nesting
    Directory.CreateDirectory(directoryPath) |> ignore

File.WriteAllText(filePath, json)